#### Execute L23

In [43]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from hashlib import sha256

from lizama_qkd import QKD

bits = 32
qkd = QKD(bits)

alice_raw_key, bob_raw_key = qkd.generate_raw_key()
print(f"Alice raw key: {alice_raw_key}")
print(f"Bob raw key: {bob_raw_key}")

alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)

assert alice_key == bob_key
print("QKD completed!")

print(f"Shared key: {alice_key}")

alice_key = sha256(alice_key.encode()).digest()
bob_key = sha256(bob_key.encode()).digest()

MSG = b"H3xTEL{R3v3rS3_R3C0nc1l14710N_1s_P34K!}"
cipher = AES.new(alice_key, AES.MODE_ECB)
encrypted = cipher.encrypt(pad(MSG, 16))

print(f"Alice encrypted message: {encrypted.hex()}")

cipher = AES.new(bob_key, AES.MODE_ECB)
decrypted = unpad(cipher.decrypt(encrypted), 16)

print(f"Bob decrypted message: {decrypted.decode()}")

Alice raw key: 10001000101010010110111101110011
Bob raw key: 10001011001100110011111101101011
QKD completed!
Shared key: 000100011100001101110101001001000100000110110010001010000001101101001111010010011010100100010010011100100110100011000110011010011111101001000001110101110011101101101111100100010010110000000000010001010111100100111000100110010101101111100011110111110000101101111100000011000000011100011011101010101111001111100100000
Alice encrypted message: aeafceee702fe7489895b510d65bf0e7cd8e7154232086c2b79cc1fcf3b44287e09ac22690c8f9214156791e182ade1f
Bob decrypted message: H3xTEL{R3v3rS3_R3C0nc1l14710N_1s_P34K!}


#### Success probability calculation

In [16]:
# preparing file
qubits = [8, 16, 32]

results = {}

for value in qubits:
    results[value] = [0, 0]
    
# file = open("success_probabilities.txt", "w")
# file.write(str(results))
# file.close()

In [24]:
def get_success_probability(bits):
    file = open("success_probabilities.txt", "r")
    results = eval(file.read())
    file.close()
    
    total = results[bits][1]
    success = results[bits][0]
    actual = 0
    for i in range(100000):
        qkd = QKD(bits)
        alice_raw_key, bob_raw_key = qkd.generate_raw_key()
        alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)
        try:
            assert alice_key == bob_key
            if alice_key:
                success += 1
                results[bits][0] += 1
            total += 1
            results[bits][1] += 1

            if (abs(actual - (success / total)) != 0) and (abs(actual - (success / total)) < 1e-4) and i > 10000:
                actual = success / total
                print(f"Success probability: {success / total}")
                break

            actual = success / total
        
            file = open("success_probabilities.txt", "w")
            file.write(str(results))
            file.close()
            
        except:
            continue

In [25]:
# 8 qubits send
get_success_probability(8)

Success probability: 0.03070095168374817


In [26]:
# 16 qubits send
get_success_probability(16)

Success probability: 0.28054389122175566


In [27]:
# 32 qubits send
get_success_probability(32)

Success probability: 0.8329334133173365


#### Tolerability

In [58]:
# preparing file
depolarize_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

results = {}

for depolarize_probability in depolarize_probabilities:
    results[depolarize_probability] = [0, 0]
    
file = open("probability_results.txt", "w")
file.write(str(results))
file.close()

In [70]:
from lizama_qkd import QKD

bits = 32
min_iterations = 5000
depolarize_probability = 0.1
rounds = 1

file = open("probability_results.txt", "r")
results = eval(file.read())
file.close()

actual_iterations = results[depolarize_probability][1]
if actual_iterations:
    actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
else:
    actual_tolerability = 0

for i in range(actual_iterations, 100000):
    alice_key = ''
    while not alice_key:
        qkd = QKD(bits, rounds = rounds, depolarize_probability = depolarize_probability)
        alice_raw_key, bob_raw_key = qkd.generate_raw_key()
        alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)
    
    if alice_key == bob_key:
        results[depolarize_probability][0] += 1
            
    results[depolarize_probability][1] += 1
    
    if (abs(actual_tolerability - (results[depolarize_probability][0] / results[depolarize_probability][1])) != 0) and (abs(actual_tolerability - (results[depolarize_probability][0] / results[depolarize_probability][1])) < 1e-4) and i > min_iterations:
        actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
        print(f"Tolerability using {depolarize_probability * 100}% error and {results[depolarize_probability][1]} iterations: {actual_tolerability}")
        break

    actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
    
    file = open("probability_results.txt", "w")
    file.write(str(results))
    file.close()

Tolerability using 10.0% error and 5002 iterations: 0.968812475009996
